<a href="https://colab.research.google.com/github/riccardomarin/EG22_Tutorial_Spectral_Geometry/blob/main/forward/01_Spectral_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will familiarize ourselves with the spectral concepts by applying them in a domain that is common in many Computer Science fields: the graphs!

The Spectral Graph theory has a long story, and here we will build some intuitions to better understand when we will move on 3D surfaces.

In [ ]:
%matplotlib inline 

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.cluster import KMeans
import cv2 

!wget https://github.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/raw/main/data/minnesota_g.mat

First of all we will build a chord Graph of 50 nodes.

In [ ]:
# 1: Building the adjacency matrix
n = 50

A = np.zeros((n, n))
a = np.array([i for i in np.arange(1,n)]) 
A[np.arange(a.size),a] = 1

# Enforcing symmetry
A = np.logical_or(A, A.T).astype(np.int32)

# It generates:
# A = np.array([  [0,1,0,0,0],
#                 [1,0,1,0,0],
#                 [0,1,0,1,0],
#                 [0,0,1,0,1],
#                 [0,0,0,1,0]])

We compute the standard Graph Laplacian; the formula is $L = D - A$, where $A$ is the adjacency matrix, and $D$ is the diagonal matrix of the degrees:

In [ ]:
# 2: Computing the Laplacian
# Vertex degree
D = np.sum(A, axis=0)
L = np.diag(D) - A

Now we can compute the spectral decomposition of the Laplacian

In [ ]:
# 3: Compute the eigenvectors
evals, evecs = np.linalg.eigh(L)

Visualizing the eigenvector and eigenvalues


In [ ]:
# 4: Plot the graph with one eigenfunction
eig_n = 1

# Create a networkx Graph object starting from our adjacency matrix
G = nx.from_numpy_matrix(A)

# Nodes displacement for visualization
pos = {i : np.asarray([i,0]) for i in np.arange(0,n)}

# Visualizing the eig_n eigenfunction
nx.draw(G, pos, node_color=evecs[:,eig_n] , node_size=40, cmap=plt.cm.bwr, vmin=np.min(evecs), vmax=np.max(evecs))
plt.show()

Eigenvalues are sorted increasingly.

In [ ]:
plt.plot(evals)

# Low Pass filtering

Since the eigenfunctions of the Laplacian provide an analougy with the Fourier Transform, we can use them as a basis to perform low-pass filtering of the functions.

We define a discontinous function.

In [ ]:
# 4: Low pass filtering

# Define a function
f = np.ones(evecs.shape[0]) * -1
f[15:25] = 1
f[40:43] = 1
nx.draw(G,pos, node_color=f, node_size=40, cmap=plt.cm.bwr, vmin=np.min(evecs), vmax=np.max(evecs))
plt.show()

Given a truncated basis $\Phi$, to perform the function low-pass the operation is: $\hat{f} = \Phi \Phi^{\dagger} f$, where $\Phi^{\dagger}$ is the pseudoinverse. 

Since in our case $\Phi$ is orthonormal, $\Phi^{\dagger} = \Phi^T$. Test the reconstruction error at varying values of k.

In [ ]:
k = 5

# Truncating the basis
evecs_trim = evecs[:,0:k]

# Synthesis and analysis with truncated basis
f_recon = np.matmul(evecs_trim, np.matmul(evecs_trim.T, f))

# Computing the reconstruction error
err = np.sqrt(np.sum(np.square(f_recon - f)))

nx.draw(G,pos, node_color=f, node_size=40, cmap=plt.cm.bwr, vmin=np.min(evecs), vmax=np.max(evecs))
plt.show()

nx.draw(G,pos, node_color=f_recon, node_size=40, cmap=plt.cm.bwr, vmin=np.min(evecs), vmax=np.max(evecs))
plt.title('Error: ' + "{:10.3f}".format(err))
plt.show()

Let's load a more complex graph: the roadmap of the Minnesota.

In [ ]:
minnesota = sio.loadmat('minnesota_g.mat')

We compute the Laplacian and its spectrum

In [ ]:
D = np.sum(minnesota['A'], axis=0)
L = np.diag(D).astype(np.int32) - minnesota['A'].astype(np.int32)

evals_min, evecs_min = np.linalg.eigh(L)

We can visualize the eigenfunctions over the graph. Notice that the first two eigenfunctions are constant! This is due to disconnected components on a graph (i.e., the multiplicity of the $0$ eigenvalue equals the number of graph components, and each $0$ eigenvalue is associated with a constant function localized on the component).

In [ ]:
G = nx.from_numpy_matrix(minnesota['A'])
pos = minnesota['pos']
k = 2

nx.draw(G,pos, node_color=evecs_min[:,k], node_size=10, cmap=plt.cm.bwr, vmin=np.min(evecs_min[:,k]), vmax=np.max(evecs_min[:,k]))
plt.show()

We can also visualize the eigenvalues, noticing how they differ from the one of the chord graph.

In [ ]:
plt.plot(evals_min)
plt.title('Eigenvalues Minnesota')